                                        ## Deep SmartGrid

In [6]:
import scipy.io
import sys

In [5]:
#caso_I.mat, caso_II.mat, caso_III.mat, caso_base.mat, consumer_data.xlsx

mat = scipy.io.loadmat('../data/caso_I.mat')

IOError: [Errno 2] No such file or directory: '../data/caso_I.mat'